In [1]:
import xport

import numpy as np
import scipy as scp

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# sns.set(color_codes=True)
%matplotlib inline

In [2]:
# open depression dataset
with open("../../../data/nhanes/2015_2016/questionaires/DPQ_I.XPT", 'rb') as f:
    depression_df = xport.to_dataframe(f)

# open demo dataset
with open("../../../data/nhanes/2015_2016/demographics/DEMO_I.XPT", 'rb') as f:
    demo_df = xport.to_dataframe(f)

# see all the output for a little bit
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

print(demo_df.shape)
print(depression_df.shape)

demo_df.head()
depression_df.head()

(9971, 47)
(5735, 11)


,SEQN,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080,DPQ090,DPQ100
0,83732.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,83733.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,83734.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,83735.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0,0.0,1.0,0.0
4,83736.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0


In [3]:
clean_depression_df = depression_df.loc[(depression_df.DPQ010 != 7) & (depression_df.DPQ010 != 9)
                                       & (depression_df.DPQ020 != 7) & (depression_df.DPQ020 != 9)
                                       & (depression_df.DPQ030 != 7) & (depression_df.DPQ030 != 9)
                                       & (depression_df.DPQ040 != 7) & (depression_df.DPQ040 != 9)
                                       & (depression_df.DPQ050 != 7) & (depression_df.DPQ050 != 9)
                                       & (depression_df.DPQ060 != 7) & (depression_df.DPQ060 != 9)
                                       & (depression_df.DPQ070 != 7) & (depression_df.DPQ070 != 9)                                                            & (depression_df.DPQ080 != 7) & (depression_df.DPQ080 != 9)
                                       & (depression_df.DPQ090 != 7) & (depression_df.DPQ090 != 9)
                                       & (depression_df.DPQ100 != 7) & (depression_df.DPQ100 != 9)]

# print(clean_depression_df.shape)
clean_depression_col_list = ['SEQN','DPQ010','DPQ020','DPQ030','DPQ040','DPQ050','DPQ060','DPQ070','DPQ080','DPQ090','DPQ100']
clean_depression_df = clean_depression_df[clean_depression_col_list].dropna()
print(clean_depression_df.shape)

# we are removign all "$20,000 and Over", "Refused, dont' know", "Under $20,000"
clean_demo_df = demo_df.loc[(demo_df.INDFMIN2 < 77) & (demo_df.INDFMIN2 != 12) & (demo_df.INDFMIN2 != 13)]
print(clean_demo_df.shape)
clean_demo_col_list = ['SEQN','INDFMIN2','DMDEDUC2']
clean_demo_df = clean_demo_df[clean_demo_col_list].dropna()
print(clean_demo_df.shape)

(3559, 11)
(8919, 47)
(5082, 3)


In [4]:
combined_dataset = pd.merge(clean_demo_df, clean_depression_df, on="SEQN")
print(combined_dataset.shape)

col_list = ['DPQ010', 'DPQ020', 'DPQ030', 'DPQ040', 'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090', 'DPQ100']
combined_dataset['DPQ_SUM'] = combined_dataset[col_list].sum(axis=1)
combined_dataset.head()

total_responders = combined_dataset.shape[0]

(3078, 13)


Depression: d (DPQ_SUM) > 9
Wealth: w (INDFMIN2) >= 45000 (8)

Education: e (DMDEDUC2) >= some college education (4)

In [5]:
import math

d_1w_1 = combined_dataset.loc[(combined_dataset.DPQ_SUM > 9) & (combined_dataset.INDFMIN2 >= 8) & (combined_dataset.DMDEDUC2 >= 4)]
d_1w_0 = combined_dataset.loc[(combined_dataset.DPQ_SUM > 9) & (combined_dataset.INDFMIN2 < 8) & (combined_dataset.DMDEDUC2 >= 4)]
d_0w_1 = combined_dataset.loc[(combined_dataset.DPQ_SUM <= 9) & (combined_dataset.INDFMIN2 >= 8) & (combined_dataset.DMDEDUC2 >= 4)]
d_0w_0 = combined_dataset.loc[(combined_dataset.DPQ_SUM <= 9) & (combined_dataset.INDFMIN2 < 8) & (combined_dataset.DMDEDUC2 >= 4)]

risk_diff_left = (len(d_1w_1)/total_responders) - (len(d_0w_1)/total_responders) 
risk_diff_right = (len(d_1w_0)/total_responders) - (len(d_0w_0)/total_responders) 
print("Interaction on Additive Scale Left Side %f" % risk_diff_left)
print("Interaction on Additive Scale Right Side %f" % risk_diff_right)


risk_diff_left_part1 = math.sqrt(((len(d_1w_1)/total_responders))*(1-(len(d_1w_1)/total_responders))/total_responders)
risk_diff_left_part2 = math.sqrt(((len(d_0w_1)/total_responders))*(1-(len(d_0w_1)/total_responders))/total_responders)

risk_diff_right_part1 = math.sqrt(((len(d_1w_0)/total_responders))*(1-(len(d_1w_0)/total_responders))/total_responders)
risk_diff_right_part2 = math.sqrt(((len(d_0w_0)/total_responders))*(1-(len(d_0w_0)/total_responders))/total_responders)

print("Risk Diff Unc Left", math.sqrt(math.pow(risk_diff_left_part1,2) + math.pow(risk_diff_left_part2,2) ))
print("Risk Diff Unc Right", math.sqrt(math.pow(risk_diff_right_part1,2) + math.pow(risk_diff_right_part2,2)))



super_additive = risk_diff_left > risk_diff_right
sub_additive = risk_diff_left < risk_diff_right
# print(super_additive)
print(sub_additive)

Interaction on Additive Scale Left Side -0.264457
Interaction on Additive Scale Right Side -0.170240
Risk Diff Unc Left 0.008653955229327307
Risk Diff Unc Right 0.008090014553382027
True


In [6]:
import math

risk_ratio_left = (len(d_1w_1)/total_responders)/(len(d_0w_0)/total_responders)
risk_ratio_right = ((len(d_1w_0)/total_responders)/(len(d_0w_0)/total_responders)) * ((len(d_0w_1)/total_responders)/(len(d_0w_0)/total_responders))

print("Interaction on Multiplicative Scale Left Side %f" % risk_ratio_left)
print("Interaction on Multiplicative Scale Right Side %f" % risk_ratio_right)


risk_ratio_left_part1 = math.sqrt(((len(d_1w_1)/total_responders))*(1-(len(d_1w_1)/total_responders))/total_responders)
risk_ratio_left_part2 = math.sqrt(((len(d_0w_0)/total_responders))*(1-(len(d_0w_0)/total_responders))/total_responders)

risk_ratio_right_part1 = math.sqrt(((len(d_1w_0)/total_responders))*(1-(len(d_1w_0)/total_responders))/total_responders)
risk_ratio_right_part2 = math.sqrt(((len(d_0w_0)/total_responders))*(1-(len(d_0w_0)/total_responders))/total_responders)
risk_ratio_right_part3 = math.sqrt(((len(d_0w_1)/total_responders))*(1-(len(d_0w_1)/total_responders))/total_responders)
risk_ratio_right_part4 = math.sqrt(((len(d_0w_0)/total_responders))*(1-(len(d_0w_0)/total_responders))/total_responders)


print("Risk Ratio Unc Left", math.sqrt(math.pow(risk_ratio_left_part1,2) + math.pow(risk_ratio_left_part1,2) ))
print("Risk Ratio Unc Right", math.sqrt(math.pow(risk_ratio_right_part1,2) + math.pow(risk_ratio_right_part2,2) + math.pow(risk_ratio_right_part3,2) + math.pow(risk_ratio_right_part4,2) ))



super_multiplicative = risk_ratio_left > risk_ratio_right
sub_multiplicative = risk_ratio_left < risk_ratio_right
# print(super_multiplicative)
print(sub_multiplicative)

Interaction on Multiplicative Scale Left Side 0.121685
Interaction on Multiplicative Scale Right Side 0.254001
Risk Ratio Unc Left 0.004006083556377593
Risk Ratio Unc Right 0.013633899116227182
True


In [7]:
import math

suff_cause_left = (len(d_1w_1)/total_responders) - (len(d_0w_1)/total_responders)
suff_cause_right = (len(d_1w_0)/total_responders)

suff_cause_left_unc_part1 = math.sqrt(((len(d_1w_1)/total_responders))*(1-(len(d_1w_1)/total_responders))/total_responders)
suff_cause_left_unc_part2 = math.sqrt(((len(d_0w_1)/total_responders))*(1-(len(d_0w_1)/total_responders))/total_responders)
suff_cause_right_unc = math.sqrt(((len(d_1w_0)/total_responders))*(1-(len(d_1w_0)/total_responders))/total_responders)

total_unc_left = math.sqrt(math.pow(suff_cause_left_unc_part1,2) + math.pow(suff_cause_left_unc_part2,2))
total_unc_right = suff_cause_right_unc
print("Unc Left ", total_unc_left)
print("Unc Right ", total_unc_right)


print(suff_cause_left)
print(suff_cause_right)

print("Sufficient condition is satisfied: %s" % (suff_cause_left > suff_cause_right))

Unc Left  0.008653955229327307
Unc Right  0.003446745452926222
-0.2644574398960364
0.038011695906432746
Sufficient condition is satisfied: False
